In [1]:
from os import listdir, path
import re

import time
import numpy as np

In [2]:
BASE_DIR_PATH = path.dirname(path.abspath("__file__"))  # gdyby nie działało, usuń cudzysłów
INSTANCES_PATH = path.join(BASE_DIR_PATH, "instances")
NEGATIVE_RANDOM_PATH = path.join(INSTANCES_PATH, "negative_random")
NEGATIVE_REPETITIONS_PATH = path.join(INSTANCES_PATH, "negative_repetitions")
POSITIVE_END_ERRORS_PATH = path.join(INSTANCES_PATH, "positive_end_errors")
POSITIVE_RANDOM_PATH = path.join(INSTANCES_PATH, "positive_random")

In [3]:
negative_random_filenames = [path.join(NEGATIVE_RANDOM_PATH, f) for f in listdir(NEGATIVE_RANDOM_PATH) if path.isfile(path.join(NEGATIVE_RANDOM_PATH, f))]
negative_repetitions_filenames = [path.join(NEGATIVE_REPETITIONS_PATH, f) for f in listdir(NEGATIVE_REPETITIONS_PATH) if path.isfile(path.join(NEGATIVE_REPETITIONS_PATH, f))]
positive_end_errors_filenames = [path.join(POSITIVE_END_ERRORS_PATH, f) for f in listdir(POSITIVE_END_ERRORS_PATH) if path.isfile(path.join(POSITIVE_END_ERRORS_PATH, f))]
positive_random_filenames = [path.join(POSITIVE_RANDOM_PATH, f) for f in listdir(POSITIVE_RANDOM_PATH) if path.isfile(path.join(POSITIVE_RANDOM_PATH, f))]

In [4]:
selected_filenames = negative_random_filenames + negative_repetitions_filenames + positive_end_errors_filenames + positive_random_filenames

for filename in selected_filenames:
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]
        
        words = file.read().splitlines()
        
        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        
        print("nazwa pliku: " + inst_name)
        print("długość słowa: " + str(word_len))
        print("oryginalna liczba słów: " + str(org_words_num))
        print("oryginalna długość sekwencji: "+ str(org_seq_len))
        print("liczba błędów negatywnych: " + str(neg_errors_number))
        print("liczba błędów pozytywnych: " + str(pos_errors_number))
        print(words[:5])
        print("\n")

nazwa pliku: C:\Users\woodpecker\Politechnika Poznańska\Semestr VI\Bioinformatyka\Laboratoria\bioinf\instances\negative_random\10.500-100
długość słowa: 10
oryginalna liczba słów: 500
oryginalna długość sekwencji: 509
liczba błędów negatywnych: 100
liczba błędów pozytywnych: 0
['AAACCATCTT', 'AAAGAGGGCG', 'AAAGGATCGC', 'AAAGTAGAAC', 'AAAGTGGAGC']


nazwa pliku: C:\Users\woodpecker\Politechnika Poznańska\Semestr VI\Bioinformatyka\Laboratoria\bioinf\instances\negative_random\10.500-200
długość słowa: 10
oryginalna liczba słów: 500
oryginalna długość sekwencji: 509
liczba błędów negatywnych: 200
liczba błędów pozytywnych: 0
['AAACCATCTT', 'AAAGGATCGC', 'AAAGTAGAAC', 'AAAGTGGAGC', 'AACACAGCAG']


nazwa pliku: C:\Users\woodpecker\Politechnika Poznańska\Semestr VI\Bioinformatyka\Laboratoria\bioinf\instances\negative_random\18.200-40
długość słowa: 10
oryginalna liczba słów: 200
oryginalna długość sekwencji: 209
liczba błędów negatywnych: 40
liczba błędów pozytywnych: 0
['AACGCATCCC', 'AAGGTA

In [ ]:
#komórka Ewy

In [ ]:
#komórka Ewy

In [ ]:
#komórka Ewy

In [ ]:
#komórka Ewy

In [6]:
#komórka Tomka

#funkcja zwraca maksymalną liczbę nukleotydów, na których zazębia się sufiks nucl1 z prefiksem nucl2
#poszukiwanie poprzez coraz to mniejszą długość ciągu, porównywanie ciągów znak po znaku
def common_nucleotides_max_number_full_iterating(nucl1, nucl2):
    word_len = len(nucl1)
    result = word_len - 1       #zmienna zawierająca końcowy wynik
    
    while(result > 0):          #badanie coraz to mniejszej możliwej długości wspólnego ciągu
        i1 = word_len - result
        i2 = 0
        
        while(i1 < word_len):              #porównanie nachodzących końcówek, znak po znaku
            if(nucl1[i1] != nucl2[i2]):    #znaki się nie zgadzają
                break
            i1 += 1
            i2 += 1
        
        if(i1 == word_len):    #porównywanie ciągów zostało wykonane pozytywnie po wszystkich znakach
            break
        
        result -= 1
    
    return result


#funkcja zwraca maksymalną liczbę nukleotydów, na których zazębia się sufiks nucl1 z prefiksem nucl2
#poszukiwanie poprzez coraz to mniejszą długość ciągu, bezpośrednie porównywanie ciągów przy pomocy mechanizmu "slicing"
def common_nucleotides_max_number_iterating_with_slicing(nucl1, nucl2):
    word_len = len(nucl1)
    result = word_len - 1       #zmienna zawierająca końcowy wynik
    
    while(result > 0):          #badanie coraz to mniejszej możliwej długości wspólnego ciągu
        i1 = word_len - result
        i2 = 0
        
        if(nucl1[-result:] == nucl2[:result]): #porównywanie ciągów przy pomocy mechanizmu "slicing"
            break
        
        result -= 1
    
    return result





#---------------- Testowanie szybkości wyznaczania wspólego sufiksu nucl1 z prefiksem nucl2 ----------------#
def speed_test(function, repetition):
    print("Function testing in progress...")
    start_time = time.time()
    filename = negative_random_filenames[0]
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]

        words = file.read().splitlines()

        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        words_num = len(words)
        
        while(repetition > 0):
            for i in range(words_num):
                for j in range(words_num):
                    if(i != j):
                        number = function(words[i], words[j])
            repetition -= 1
    
    elapsed_time = time.time() - start_time
    print("End of the function testing.")
    print("Time: " + str(elapsed_time) + "\n")

#Wywołanie testowania funkcji
#speed_test(common_nucleotides_max_number_full_iterating, 10)            #repetition=10 Time: 13.98816466331482
#speed_test(common_nucleotides_max_number_iterating_with_slicing, 10)    #repetition=10 Time: 16.22281813621521

Function testing in progress...
End of the function testing.
Time: 13.98816466331482

Function testing in progress...
End of the function testing.
Time: 16.22281813621521



In [7]:
#komórka Tomka
filename = negative_random_filenames[0]
with open(filename, 'r') as file:
    inst_name = filename[filename.rfind('/') + 1:]

    words = file.read().splitlines()

    word_len = len(words[0])
    org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
    org_seq_len = org_words_num + word_len - 1
    neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
    neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
    pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
    pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
    words_num = len(words)

    print("nazwa pliku: " + inst_name)
    print("długość słowa: " + str(word_len))
    print("oryginalna liczba słów: " + str(org_words_num))
    print("oryginalna długość sekwencji: "+ str(org_seq_len))
    print("liczba błędów negatywnych: " + str(neg_errors_number))
    print("liczba błędów pozytywnych: " + str(pos_errors_number))
    print("liczba oligonukleotydów: " + str(words_num))
    print(words[:5])
    print("\n")
    
    print("common_nucleotides_max_number_full_iterating\tstart")
    for i in range(words_num):
        for j in range(words_num):
            if(i != j):
                number = common_nucleotides_max_number_full_iterating(words[i], words[j])
                #print(words[0])
                #print((word_len - number) * ' ' + words[i] + "\tnumber=" + str(number) + "\n")
    print("common_nucleotides_max_number_full_iterating\tstop")

nazwa pliku: C:\Users\woodpecker\Politechnika Poznańska\Semestr VI\Bioinformatyka\Laboratoria\bioinf\instances\negative_random\10.500-100
długość słowa: 10
oryginalna liczba słów: 500
oryginalna długość sekwencji: 509
liczba błędów negatywnych: 100
liczba błędów pozytywnych: 0
liczba oligonukleotydów: 400
['AAACCATCTT', 'AAAGAGGGCG', 'AAAGGATCGC', 'AAAGTAGAAC', 'AAAGTGGAGC']


common_nucleotides_max_number_full_iterating	start
common_nucleotides_max_number_full_iterating	stop


In [34]:
#komórka Tomka

In [ ]:
#komórka Tomka